In [ ]:
import pandas as pd
import numpy as np
fn = '../src/data/raw/Community_2005-2017_Greenhouse_Gas_Emissions.csv'

emissions = pd.read_csv(fn)
emissions

In [ ]:
emissions_trunc = pd.DataFrame(data=emissions.iloc[0:11])
emissions_trunc = pd.concat([emissions_trunc, emissions.iloc[13:14]], ignore_index=True)
emissions_trunc.index = emissions_trunc['Unnamed Column'].to_list()
emissions_trunc.index.names = ['Source']
emissions_trunc = emissions_trunc.replace(',','', regex=True).fillna(value=0)
emissions_trunc = emissions_trunc.apply(pd.to_numeric, errors='coerce')
emissions_trunc.iloc[1]= emissions_trunc.iloc[1] + emissions_trunc.iloc[2]
emissions_trunc.iloc[3]= emissions_trunc.iloc[3] + emissions_trunc.iloc[9]
emissions_trunc.iloc[5]= (emissions_trunc.iloc[5] + 
                          emissions_trunc.iloc[6] + 
                          emissions_trunc.iloc[7])
small_row_names = ['Large Commercial Electricity', 
                   'Commercial Gas Consumption', 
                   'Fugitive Gas Emissions', 
                   'CTA Rail Electricty']
emissions_trunc = emissions_trunc.drop(small_row_names)
rename_dict = {'Small Commercial Electricity':'Commercial Electricity',
              'Industrial Gas Consumption': 'Non-Residential Gas Consumption'}
emissions_trunc = emissions_trunc.rename(rename_dict).iloc[:,1:]
emissions_trunc

In [ ]:
emissions_trunc.to_csv('2005-2017_Truncated.csv')

In [ ]:
trunc_emissions = pd.read_csv('2005-2017_Truncated.csv')
year_str = [str(int(blah)) for blah in np.linspace(2005, 2017, 13)]
idx_list = trunc_emissions.index
# idx_list
non_totals = trunc_emissions['Source'].copy()
non_totals[3] = 'Residential Gas'
non_totals[4] = 'Non-Residential Gas'
non_totals[5] = 'Mixed Vehicular Combined'
non_totals[6] = 'Mixed Refuse Biomass'
non_totals[7] = 'Municipal Combined'
type_list = [list(non_totals.values),
             [non_totals.str.split(' ')[i][-1].strip() for i in idx_list],
             [non_totals.str.split(' ')[i][0].strip() for i in idx_list]]
idx = pd.MultiIndex.from_arrays(type_list, names=['original', 'source', 'use'])
len_columns = [len(trunc_emissions.columns)]
emissions_lvld = trunc_emissions.copy()
emissions_lvld.index = idx
emissions_lvld.loc[:, 'id'] = np.array(np.arange(len(emissions_lvld)))
emissions_lvld.loc[:, 'source'] = type_list[1]
emissions_lvld.loc[:, 'use'] = type_list[2]
emissions_long = emissions_lvld.melt(col_level=0, id_vars=['id', 'source', 'use'], value_vars=year_str, var_name='year')
emissions_long['id'] = emissions_long.index
emissions_long

In [ ]:
len_data = len(emissions_long)
interval = 8
# link_data = np.array([[int(blah) for blah in np.linspace(0,len_data-interval,len_data-interval+1)], 
#                       [int(blah) for blah in np.linspace(interval,len_data,len_data-interval+1)],
#                       emissions_long['value'].values[0:len_data-interval]]).T
link_data = np.transpose([emissions_long['id'].values[0:len_data-interval],
                          emissions_long['id'].values[interval:len_data],
                          emissions_long['value'].values[interval:len_data]])
link_df = pd.DataFrame(link_data, columns = ['source_id','target_id', 'strength'], dtype=object)
link_df

In [ ]:
emissions_long.to_csv('emissions2017_long.csv')
link_df.to_csv('emissions2017_link.csv')